<a href="https://colab.research.google.com/github/Chirag314/ICR-age-related-conditions/blob/main/ICR_Age_related_conditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
#from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from datetime import date, datetime

import warnings
warnings.filterwarnings("ignore")

Ref:--https://www.kaggle.com/general/74235

How to copy kaggle datasets to colab

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle # Remove if dir exists
!mkdir ~/.kaggle # make directory
!cp kaggle.json ~/.kaggle # copy json file to dir.

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json #change the access mode

In [ ]:
!kaggle datasets list # list all datasets

In [ ]:
#download data
! kaggle competitions download -c icr-identify-age-related-conditions

In [ ]:
!rm -r train
! mkdir dataset
!cp /content/icr-identify-age-related-conditions.zip /content/dataset/icr-identify-age-related-conditions.zip
!rm /content/dataset/icr-identify-age-related-conditions.zip.

In [ ]:
!unzip -q /content/dataset/icr-identify-age-related-conditions.zip -d /content/dataset/

In [ ]:
! unzip icr-identify-age-related-conditions dataset\

Iterative stratified K fold
ref:https://www.kaggle.com/code/sambitmukherjee/multilabel-stratified-k-fold-creation

In [ ]:
!pip install iterative-stratification

In [ ]:
import sys
#sys.path.append('../input/iterativestratification')
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
!pip install tabpfn

In [ ]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
#!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

!cp /user/local/lib/python3.10/dist-packages/tabpfn/models_diff/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/


In [ ]:
from tabpfn import TabPFNClassifier

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import FastICA
from sklearn.decomposition import NMF

### PARAMS

In [ ]:
PATH = '../content/dataset'
NUMCOLS = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ',
           'BC', 'BD', 'BN', 'BP', 'BQ', 'BR', 'BZ',
           'CB', 'CC', 'CD', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU', 'CW',
           'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
           'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU',
           'FC', 'FD', 'FE', 'FI', 'FL', 'FR', 'FS',
           'GB', 'GE', 'GF', 'GH', 'GI', 'GL',
           'Epsilon']
CATCOLS = ['EJ']
#
NFOLDS = 20
#
CHECK = [(1,1.,0.), (2,1.,0.)]

### FUNCTIONS

In [ ]:
def competition_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2
#

In [ ]:
def loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1
#=======
def best_threshold(y_true, y_pred, metrics, n_breaks=101):
    bt = None
    bs = -1
    proposals = np.linspace(0,1,n_breaks)
    for proposal in proposals:
        cur = metrics(y_true, (y_pred>proposal).astype(int))
        if cur>bs:
            bs = cur
            bt = proposal
        #
    #
    return bt, bs
#=======

In [ ]:
def pp_prob(p):
    c0 = p[:,0].sum()
    c1 = p[:,1:].sum()
    new_p = p * np.array([[1/(c0 if i==0 else c1) for i in range(p.shape[1])]])
    new_p = new_p / np.sum(new_p,axis=1,keepdims=1)
    return np.sum(new_p[:,1:],1,keepdims=False)
#====================
def train_skl(x, y, xe, folds, how='log'):
    n_folds = len(folds)
    oof = np.zeros((len(y), ))
    preds = np.zeros((len(xe),))

    print('='*30)
    for idx in range(n_folds):
        print("FOLD:", idx)
        tr_idx, val_idx = folds[idx]
        xt, yt = x[tr_idx], y[tr_idx]
        xv, yv = x[val_idx], y[val_idx]
        #t_w0, t_w1 = loss_weight(yt)
        #v_w0, v_w1 = loss_weight(yv)
        #wt = pd.DataFrame(yt, columns=['0'])['0'].map({0:t_w0,1:t_w1}).values
        #wv = pd.DataFrame(yv, column['0'])['0'].map({0:v_w0,1:v_w1}).values

        t_w0, t_w1 = loss_weight((yt!=0).astype(int))
        v_w0, v_w1 = loss_weight((yv!=0).astype(int))
        wt = pd.DataFrame((yt!=0).astype(int), columns=['0'])['0'].map({0:t_w0,1:t_w1}).values
        wv = pd.DataFrame((yv!=0).astype(int), columns=['0'])['0'].map({0:v_w0,1:v_w1}).values

        if how == 'log':
            model = LogisticRegression()
        elif how == 'rf':
            model = RandomForestClassifier(n_estimators=100, max_depth=4)
        elif how == 'svc':
            model = SVC(gamma='auto', probability=True, C=1.)
        elif how == 'knn':
            model = KNeighborsClassifier(n_neighbors=50)
        elif how == 'xgb':
            model = xgb.XGBClassifier(max_depth=3)
        elif how =='lgb':
            model = lgb.LGBMClassifier(max_depth=3)
        elif how =='tabfpn':
            model = TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')
        elif how == 'mlp':
            model = MLPClassifier(hidden_layer_sizes=(100,100,),
                                  random_state=777, max_iter=300)
        elif how == 'tree':
            model = DecisionTreeClassifier(max_depth=5)
        elif how == 'qda':
            model = QuadraticDiscriminantAnalysis()
        else:
            model = AdaBoostClassifier( RandomForestClassifier(n_estimators=100, max_depth=4) )
        #
        model.fit(xt, yt)
        """
        if how in ['svc','mlp','knn', 'qda', 'tabfpn']:
            model.fit(xt, yt)
        else:
            model.fit(xt, yt, sample_weight=wt)
        """
        #
        oof[val_idx] =  pp_prob( model.predict_proba(xv) )
        preds += pp_prob( model.predict_proba(xe) )/n_folds
        #
        print('='*30)
    return oof, preds
#===================


### TRAINING

In [ ]:
def preprocess(df):
    df['EJ'] = df['EJ'].map({'A':0,'B':1})
    df = df.rename(columns={'BD ': 'BD', 'CD ': 'CD', 'CW ': 'CW', 'FD ': 'FD'})
    return df

In [ ]:
tr = pd.read_csv(f'{PATH}/train.csv')
te = pd.read_csv(f'{PATH}/test.csv')
tr = preprocess(tr)
te = preprocess(te)

greeks = pd.read_csv(f'{PATH}/greeks.csv')
greeks.loc[greeks.Epsilon != 'Unknown','Epsilon'] = greeks.loc[greeks.Epsilon != 'Unknown','Epsilon'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
greeks.loc[greeks.Epsilon == 'Unknown','Epsilon'] = np.nan
greeks['Alpha'] = greeks['Alpha'].map({'A':0,'B':1,'D':2,'G':3})
tr = tr.merge(greeks, on='Id')
te['Epsilon'] = tr['Epsilon'].max() + 1
##

In [ ]:
tr.head()

In [ ]:
X = tr[NUMCOLS + CATCOLS].values
y = tr['Class'].values
ym = tr['Alpha'].values
Xe = te[NUMCOLS + CATCOLS].values
Z = greeks.iloc[:,1:-1].values

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp.fit_transform(X)
Xe = imp.transform(Xe)

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)
Xe = sc.transform(Xe)

In [ ]:
X.mean(), Xe.mean(),

In [ ]:
pca = PCA(n_components=10, svd_solver='full', whiten=True)
X_pca = pca.fit_transform(X)
Xe_pca = pca.transform(Xe)

In [ ]:
lda = LDA(n_components=1)
X_lda = lda.fit_transform(X, y)
Xe_lda = lda.transform(Xe)

In [ ]:
#tsne = TSNE(n_components=3)
#X_tsne = tsne.fit_transform(X)
#Xe_tsne = tsne.fit_transform(Xe)

In [ ]:
agglo = FeatureAgglomeration(n_clusters=10)
X_agglo = agglo.fit_transform(X)
Xe_agglo = agglo.transform(Xe)

In [ ]:
transformer = FastICA(n_components=10, random_state=0)
X_fast = transformer.fit_transform(X)
Xe_fast = transformer.transform(Xe)

In [ ]:
move = min(X.min(), Xe.min())
model = NMF(n_components=10, init='random', random_state=0)
X_nmf = model.fit_transform(X - move)
Xe_nmf = model.transform(Xe - move)

In [ ]:
X_agglo.shape, X_nmf.shape

In [ ]:
pca.explained_variance_.sum()

In [ ]:
#HOW = 'lgb'
#oof1, p1 = train_skl(X, ym, Xe, FOLDS, how=HOW)
#oof1, p1 = train_skl(X_pca, y, Xe_pca, FOLDS, how=HOW)
#oof1, p1 = train_skl(X_lda, y, Xe_lda, FOLDS, how=HOW)
#oof1, p1 = train_skl(X_agglo, y, Xe_agglo, FOLDS, how=HOW)
#oof1, p1 = train_skl(X_fast, y, Xe_fast, FOLDS, how=HOW)
#oof1, p1 = train_skl(X_nmf, y, Xe_nmf, FOLDS, how=HOW)

In [ ]:
#skf = StratifiedKFold(n_splits=NFOLDS)
#FOLDS = list(skf.split(X, y))

skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
FOLDS = list(skf.split(X, Z))

In [ ]:
oof1, p1 = train_skl(X, ym, Xe, FOLDS, how='lgb')
oof2, p2 = train_skl(X, ym, Xe, FOLDS, how='xgb')
oof3, p3 = train_skl(X, ym, Xe, FOLDS, how='tabfpn')

In [ ]:
print('LGB', competition_log_loss(y, oof1))
print('XGB', competition_log_loss(y, oof2))
print('TabFPN', competition_log_loss(y, oof3))
print('Blend', competition_log_loss(y, 0.6*oof2 + 0.4*oof3))

In [ ]:
oof1.max(), oof1.min(), p1.max(), p1.min()

In [ ]:
sub = te[['Id']].copy()
p = 0.5 * p2 + 0.5 * p3
sub['class_0'] = 1. - p
sub['class_1'] = p
#===========
#===========
sub.to_csv('submission.csv', index=False)
sub.head()